<h1><strong>Cheatsheet</strong>: brincando com a API Python do Blender.</h1>

<h2>SNIPPETS DE UMA LINHA</h2>

pega o objeto atualmente sendo editado.
Se nao estiver em modo de edição, não retorna nada.

In [ ]:
bpy.context.edit_object

Troca entre modo de edição ativado e desativado.

In [ ]:
bpy.ops.object.editmode_toggle()

A localização de um objeto:

In [ ]:
bpy.data.objects["Cube"].delta_location #pode somar para modificar!

A seguir sobre mexer com os atributos de objetos:

In [ ]:
bpy.data.scenes #pega as cenas
bpy.data.scenes[0].objects #pega os objetos de uma cena
bpy.data.scenes[0].objects[0].data #aqui pode acessar os atributos de um objeto.
#Por exemplo:
bpy.data.scenes[0].objects[0].data.vertices # é uma lista de todos vértices do objeto.
bpy.data.scenes[0].objects[0].data.vertices[0].co # são as coordenadas de um dos vértices.
#Também dá pra fazer:
bpy.data.scenes[0].objects[0].data.vertices[0].co.x #.y ou .z também

Observação: mexer com a coordenada de um vértice só funciona se não estiver em edit_mode.

In [ ]:
bpy.data.scenes[0].objects[0].data.vertices[0].co += mathutils.Vector((0,0,2))

Criação e deleção de objetos é feita assim:

In [ ]:
mesh = bpy.data.meshes.new(name="MyMesh")
bpy.data.meshes.remove(mesh)

Como deletar um objeto específico:

In [ ]:
# seleção
bpy.data.objects['Cube'].select = True

# remoção
bpy.ops.object.delete()

<h2>SNIPPETS DE CORPO</h2>

Selecionar vertices de uma mesh

In [ ]:
import bpy,bmesh

ob = bpy.data.objects['wave']
bpy.context.scene.objects.active = ob
bpy.ops.object.mode_set(mode='EDIT')
mesh=bmesh.from_edit_mesh(bpy.context.object.data)
for v in mesh.verts:
    v.select = True

# trigger viewport update
bpy.context.scene.objects.active = bpy.context.scene.objects.active

Como criar uma mesh, acrescentar vértices e torná-la um objeto:

In [ ]:
import bmesh

verts = [(1, 1, 1), (0.5, 0.5, 0.5)]  # 2 verts made with XYZ coords
mesh = bpy.data.meshes.new("mesh")  # add a new mesh
obj = bpy.data.objects.new("MyObject", mesh)  # add a new object using the mesh

scene = bpy.context.scene
scene.objects.link(obj)  # put the object into the scene (link)
scene.objects.active = obj  # set as the active object in the scene
obj.select = True  # select object

mesh = bpy.context.object.data
bm = bmesh.new()

for v in verts:
    bm.verts.new(v)  # add a new vert

# make the bmesh the object's mesh
bm.to_mesh(mesh)  
bm.free()  # always do this when finished

Como acrescentar vértices a uma mesh já existente:

In [ ]:
import bmesh

verts = [(1, 1, 1), (0, 0, 0)]  # 2 verts made with XYZ coords
mesh = bpy.context.object.data
bm = bmesh.new()

# convert the current mesh to a bmesh (must be in edit mode)
bpy.ops.object.mode_set(mode='EDIT')
bm.from_mesh(mesh)
bpy.ops.object.mode_set(mode='OBJECT')  # return to object mode

for v in verts:
    bm.verts.new(v)  # add a new vert

# make the bmesh the object's mesh
bm.to_mesh(mesh)  
bm.free()  # always do this when finished

Criar mesh, transformar em objeto, criar vértices nesse objeto e depois fechar uma face:

In [ ]:
import bmesh

mesh = bpy.data.meshes.new("mesh")  # add a new mesh
obj = bpy.data.objects.new("MyObject", mesh)  # add a new object using the mesh

scene = bpy.context.scene
scene.objects.link(obj)  # put the object into the scene (link)
scene.objects.active = obj  # set as the active object in the scene
obj.select = True  # select object

mesh = bpy.context.object.data

#PRECISA ESTAR EM EDIT MODE:
bpy.ops.object.mode_set(mode='EDIT')

bm = bmesh.from_edit_mesh(mesh)

v1 = bm.verts.new((2.0, 2.0, 2.0))
v2 = bm.verts.new((-2.0, 2.0, 2.0))
v3 = bm.verts.new((-2.0, -2.0, 2.0))

bm.faces.new((v1, v2, v3))

bmesh.update_edit_mesh(obj.data)

#VOLTANDO AO OBJ MODE:
bpy.ops.object.mode_set(mode='OBJECT')

<strong>SUPERFÍCIE EM ONDA</strong>

In [ ]:
import bpy, math
# mesh arrays
verts = []  # the vertex array
faces = []  # the face array
 
# mesh variables
numX = 10  # number of quadrants in the x direction
numY = 10  # number of quadrants in the y direction
 
# wave variables
freq = 1  # the wave frequency
amp = 1  # the wave amplitude
scale = 1  #the scale of the mesh

#fill verts array
for i in range (0, numX):
    for j in range(0,numY):
        x = scale * i
        y = scale * j
        z = scale*((amp*math.cos(i*freq))+(amp*math.sin(j*freq)))
        vert = (x,y,z) 
        verts.append(vert)
 
#create mesh and object
mesh = bpy.data.meshes.new("wave")
object = bpy.data.objects.new("wave",mesh)
 
#set mesh location
object.location = bpy.context.scene.cursor_location
bpy.context.scene.objects.link(object)
 
#create mesh from python data
mesh.from_pydata(verts,[],faces)
mesh.update(calc_edges=True)